In [1]:
import torch
import numpy as np
import random
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

DATA_DIR = "./data"

# 시드 번호 고정
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# 1) 이미지 전처리(아직은 심플하게)
train_tf = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

valid_tf = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

In [3]:
# 2) ImageFolder로 데이터셋 생성
train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "train"),
                                transform=train_tf)
valid_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "valid"),
                                transform=valid_tf)

In [4]:
# 3) DataLoader 만들기 (윈도우면 num_workers=0부터 시작 추천)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=False, num_workers=0)

In [5]:
# 4) 정보 출력해서 제대로 읽히는지 확인
print("클래스 인덱스 매핑:", train_ds.class_to_idx)
print("train 이미지 개수:", len(train_ds))
print("valid 이미지 개수:", len(valid_ds))

images, labels = next(iter(train_loader))
print("한 배치 이미지 텐서 크기:", images.shape)  # [B, C, H, W] 예상
print("한 배치 라벨:", labels[:10])

클래스 인덱스 매핑: {'건선': 0, '아토피': 1, '여드름': 2, '정상': 3, '주사': 4, '지루': 5}
train 이미지 개수: 4800
valid 이미지 개수: 600
한 배치 이미지 텐서 크기: torch.Size([32, 3, 224, 224])
한 배치 라벨: tensor([2, 2, 3, 0, 3, 2, 5, 5, 1, 0])


In [6]:
# !pip install mlflow
import mlflow
# import mlflow.pytorch  # 나중에 모델 저장에 사용

# 로컬 디렉터리에 결과 저장 (기본값이지만 명시해두면 좋음)
mlflow.set_tracking_uri("file:./mlruns")

# 실험 이름 설정
mlflow.set_experiment("skin_disease_cnn_vit")

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
2025/11/18 14:51:21 INFO mlflow.tracking.fluent: Experiment with name 'skin_disease_cnn_vit' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/SSAFY/Downloads/ssafy_1th/mlruns/195089595728592597', creation_time=1763445081803, experiment_id='195089595728592597', last_update_time=1763445081803, lifecycle_stage='active', name='skin_disease_cnn_vit', tags={}>

In [7]:
import random
import time

with mlflow.start_run(run_name="dummy_test"):
    # 하이퍼파라미터 같은 것들
    mlflow.log_param("model_type", "dummy")
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("num_epochs", 3)

    # 에폭 3번 도는 척하면서 loss/acc를 찍어보기
    for epoch in range(1, 4):
        # 여기서는 그냥 랜덤 숫자 (나중에 진짜 train/valid 값으로 교체)
        train_loss = random.uniform(0.5, 1.0)
        val_loss = random.uniform(0.5, 1.0)
        train_acc = random.uniform(0.6, 0.9)
        val_acc = random.uniform(0.6, 0.9)

        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("train_acc", train_acc, step=epoch)
        mlflow.log_metric("val_acc", val_acc, step=epoch)

        print(f"epoch {epoch} | train_loss={train_loss:.3f}, val_loss={val_loss:.3f}")

        time.sleep(0.5)  # 로그 찍히는 느낌 보려고 살짝 대기


epoch 1 | train_loss=0.820, val_loss=0.513
epoch 2 | train_loss=0.868, val_loss=0.838
epoch 3 | train_loss=0.711, val_loss=0.515


In [8]:
import mlflow
import pandas as pd

# 트래킹 URI 맞춰주기 (지금 코드와 동일하게)
mlflow.set_tracking_uri("file:./mlruns")

# 내가 만든 실험 이름
experiment_name = "skin_disease_cnn_vit"

# 이 실험에 해당하는 run들을 전부 DataFrame으로 가져오기
df = mlflow.search_runs(experiment_names=[experiment_name])

# 전체 확인
df.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.train_loss,metrics.train_acc,metrics.val_acc,metrics.val_loss,params.batch_size,params.num_epochs,params.model_type,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.type
0,3c0b5a4b2d864da08c080e7c222ce8fe,195089595728592597,FINISHED,file:///c:/Users/SSAFY/Downloads/ssafy_1th/mlr...,2025-11-18 05:51:21.929000+00:00,2025-11-18 05:51:23.475000+00:00,0.710961,0.665591,0.751607,0.514899,32,3,dummy,C:\Users\SSAFY\AppData\Roaming\Python\Python31...,SSAFY,dummy_test,LOCAL
